In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder \
    .appName("Train_MTA_RF_Model") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

df = spark.read \
    .format("mongo") \
    .option("uri", "mongodb://localhost:27017/mta_db.foot_traffic") \
    .load()


:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/gopalakrishnaabba/.ivy2/cache
The jars for the packages stored in: /Users/gopalakrishnaabba/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d9451df7-65a1-4864-b7c0-5b2f791c33f7;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 117ms :: artifacts dl 5ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules   

In [3]:

# Index and assemble features
indexer = StringIndexer(inputCol="STATION", outputCol="station_index")
assembler = VectorAssembler(inputCols=["station_index"], outputCol="features")

# # Model for ENTRIES
# rf_entries = RandomForestRegressor(featuresCol="features", labelCol="sum(ENTRIES)")
# pipeline_entries = Pipeline(stages=[indexer, assembler, rf_entries])
# model_entries = pipeline_entries.fit(df)
# model_entries.write().overwrite().save("/Users/gopalakrishnaabba/mta_rf_entries_model")

# # Model for EXITS
# rf_exits = RandomForestRegressor(featuresCol="features", labelCol="sum(EXITS)")
# pipeline_exits = Pipeline(stages=[indexer, assembler, rf_exits])
# model_exits = pipeline_exits.fit(df)
# model_exits.write().overwrite().save("/Users/gopalakrishnaabba/mta_rf_exits_model")

In [4]:
# Model for ENTRIES
rf_entries = RandomForestRegressor(featuresCol="features", labelCol="sum(ENTRIES)")
pipeline_entries = Pipeline(stages=[indexer, assembler, rf_entries])
model_entries = pipeline_entries.fit(df)

# Print model summary for ENTRIES
rf_model_entries = model_entries.stages[-1]
print("ENTRIES Model:")
print(f" - Number of Trees: {rf_model_entries.getNumTrees}")
print(f" - Feature Importances: {rf_model_entries.featureImportances}")
print(f" - Tree Weights: {rf_model_entries.treeWeights}")

# Save model
model_entries.write().overwrite().save("/Users/gopalakrishnaabba/mta_rf_entries_model")

# Model for EXITS
rf_exits = RandomForestRegressor(featuresCol="features", labelCol="sum(EXITS)")
pipeline_exits = Pipeline(stages=[indexer, assembler, rf_exits])
model_exits = pipeline_exits.fit(df)

# Print model summary for EXITS
rf_model_exits = model_exits.stages[-1]
print("\nEXITS Model:")
print(f" - Number of Trees: {rf_model_exits.getNumTrees}")
print(f" - Feature Importances: {rf_model_exits.featureImportances}")
print(f" - Tree Weights: {rf_model_exits.treeWeights}")

# Save model
model_exits.write().overwrite().save("/Users/gopalakrishnaabba/mta_rf_exits_model")


25/05/04 17:36:55 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 32 to 10 (= number of training instances)


ENTRIES Model:
 - Number of Trees: 20
 - Feature Importances: (1,[0],[1.0])
 - Tree Weights: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


25/05/04 17:36:57 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 32 to 10 (= number of training instances)



EXITS Model:
 - Number of Trees: 20
 - Feature Importances: (1,[0],[1.0])
 - Tree Weights: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
